In [ ]:
import pandas as pd
import os

# 1. TSV 파일이 포함된 폴더에서 모든 파일을 읽어오는 함수
def load_tsv_files_from_folder(folder_path="C:/python/python(1-0)TSV"):
    # 폴더 내의 모든 .tsv 파일을 찾음
    tsv_files = [f for f in os.listdir(folder_path) if f.endswith('.tsv')]
    return tsv_files

# 2. TSV 파일 읽기 및 데이터 처리
def process_tsv_file(file_path, output_csv="(1-1)모든 transactions.csv"):
    # 2. TSV 파일 읽기
    df = pd.read_csv(file_path, sep='\t')

    # 3. 'input_total' 열이 있는지 확인 후, 필터링
    if 'input_total' in df.columns:
        # 'input_total'이 문자열로 되어있다면, 이를 숫자로 변환 (단위는 Satoshis)
        df['input_total'] = pd.to_numeric(df['input_total'], errors='coerce')

        # Satoshis를 BTC로 변환 (1 BTC = 100,000,000 satoshis)
        df['input_total'] = df['input_total'] / 1e8  # Satoshi -> BTC

        # 3-1. 10 BTC 미만인 행을 삭제
        df = df[df['input_total'] >= 10]
    else:
        print(f"파일에 'input_total' 컬럼이 없습니다: {file_path}")
        return None

    # 4. hash와 time 데이터를 제외한 나머지 모든 데이터 삭제
    if 'hash' in df.columns and 'time' in df.columns:
        df = df[['hash', 'time']]  # 'hash'와 'time' 열만 남기고 나머지 삭제
    else:
        missing_columns = [col for col in ['hash', 'time'] if col not in df.columns]
        print(f"파일에 다음 컬럼이 없습니다: {', '.join(missing_columns)}")
        return None

    # 5. 트랜잭션 hash와 time 데이터를 '현물 large_transactions-1.csv'에 추가 저장
    if not os.path.exists(output_csv):  # 파일이 없으면 헤더 포함
        df.to_csv(output_csv, index=False, mode='w', encoding='utf-8')
    else:  # 파일이 있으면 헤더 제외
        df.to_csv(output_csv, index=False, mode='a', header=False, encoding='utf-8')

    print(f"{output_csv} 파일에 트랜잭션 hash와 time 데이터를 추가 저장했습니다.")

# 6. 폴더 내 모든 TSV 파일 처리
def process_files_in_folder(folder_path="C:/python/python(1-0)TSV"):
    # 폴더 내 모든 .tsv 파일을 찾음
    tsv_files = load_tsv_files_from_folder(folder_path)

    for file in tsv_files:
        file_path = os.path.join(folder_path, file)
        print(f"파일 처리 중: {file}")

        # TSV 파일 처리
        process_tsv_file(file_path)

# 실행
process_files_in_folder("C:/python/python(1-0)TSV")


In [ ]:
from binance.client import Client
import pandas as pd
import time

# API 키와 Secret 키 입력
api_key = "YOUR_API_KEY"
api_secret = "YOUR_API_SECRET"

# 바이낸스 클라이언트 생성
client = Client(api_key, api_secret)

# 데이터 가져오기 함수
def get_historical_15m_data(symbol, start_date, end_date, interval='15m'):
    """
    바이낸스에서 과거 15분 봉 데이터를 가져와 DataFrame으로 반환
    :param symbol: 거래 페어 (예: "BTCUSDT")
    :param start_date: 시작 날짜 (예: "2023-01-01")
    :param end_date: 종료 날짜 (예: "2025-01-31")
    :param interval: 데이터 간격 (기본값: '15m')
    :return: Pandas DataFrame
    """
    # 바이낸스 API의 제한으로 인해 데이터를 나눠서 요청해야 함
    all_klines = []
    while True:
        # 데이터 요청
        klines = client.get_historical_klines(symbol, interval, start_date, end_date)
        if not klines:
            break

        all_klines.extend(klines)

        # 데이터 끝났는지 확인
        last_timestamp = klines[-1][0]
        if last_timestamp >= pd.Timestamp(end_date).timestamp() * 1000:
            break

        # 다음 시작 날짜 설정
        start_date = pd.Timestamp(last_timestamp, unit='ms') + pd.Timedelta(seconds=1)

        # API 요청 제한을 방지하기 위해 대기
        time.sleep(0.2)

    # 데이터프레임 생성 및 정리
    columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume',
               'close_time', 'quote_asset_volume', 'number_of_trades',
               'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore']

    df = pd.DataFrame(all_klines, columns=columns)
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df = df[['timestamp', 'open', 'high', 'low', 'close', 'volume']]
    df[['open', 'high', 'low', 'close', 'volume']] = df[['open', 'high', 'low', 'close', 'volume']].astype(float)

    return df

# 데이터 가져오기
symbol = "BTCUSDT"
start_date = "2023-01-01"
end_date = "2025-01-31"
interval = "15m"

print("데이터 다운로드 중...")
btc_data = get_historical_15m_data(symbol, start_date, end_date, interval)
print("데이터 다운로드 완료!")

# 데이터 저장
output_path = r"C:\python\BTC_15m_data.csv"
btc_data.to_csv(output_path, index=False)
print(f"BTC 15분 봉 데이터가 '{output_path}'에 저장되었습니다.")


In [ ]:
import os
import pandas as pd

# 파일 경로 설정
input_file = r"C:\python\BTC_15m_data.csv"
output_file = r"C:\python\BTC_15m_data-datetime.csv"

# 파일 읽기 및 처리
if os.path.exists(input_file):
    # CSV 파일 읽기
    btc_prices_df = pd.read_csv(input_file)

    # 'timestamp' 열을 datetime 형식으로 변환
    try:
        btc_prices_df['timestamp'] = pd.to_datetime(btc_prices_df['timestamp'])
        print("'timestamp' 열이 24시간제 datetime 형식으로 변환되었습니다.")

        # 새로운 파일로 저장
        btc_prices_df.to_csv(output_file, index=False)
        print(f"새로운 파일로 저장되었습니다: {output_file}")
    except Exception as e:
        print("'timestamp' 열을 datetime 형식으로 변환할 수 없습니다.")
        print("오류:", e)
else:
    print(f"파일이 존재하지 않습니다: {input_file}")


In [ ]:
import pandas as pd
import os
import requests
from concurrent.futures import ThreadPoolExecutor

# Blockchain.com API를 사용하여 트랜잭션 데이터를 추출하는 함수
def get_transaction_inputs_outputs(txid):
    url = f"https://blockchain.info/rawtx/{txid}"
    try:
        response = requests.get(url)

        # API 응답 상태 코드 확인
        if response.status_code != 200:
            print(f"Blockchain.com API 요청 실패: 상태 코드 {response.status_code}, txid: {txid}")
            return None

        data = response.json()

        # 입력(출금) 및 출력(입금) 데이터 처리
        inputs = [
            {'address': inp.get('prev_out', {}).get('addr', 'Unknown'),
             'value': inp.get('prev_out', {}).get('value', 0) / 1e8,  # Satoshi -> BTC
             'type': 'withdrawal'} for inp in data.get('inputs', [])
        ]

        outputs = [
            {'address': out.get('addr', 'Unknown'),
             'value': out.get('value', 0) / 1e8,  # Satoshi -> BTC
             'type': 'deposit'} for out in data.get('out', [])
        ]

        # 데이터 병합
        all_data = inputs + outputs
        balance_df = pd.DataFrame(all_data)

        # 트랜잭션 날짜 추가
        balance_df['date'] = pd.to_datetime(data.get('time', 0), unit='s')

        return balance_df

    except requests.exceptions.RequestException as e:
        print(f"네트워크 요청 오류: {e}, txid: {txid}")
        return None
    except ValueError as e:
        print(f"JSON 디코드 오류: {e}, txid: {txid}")
        return None
    except KeyError as e:
        print(f"데이터 키 오류: {e}, txid: {txid}")
        return None

# 데이터를 CSV 파일에 저장하는 함수
def save_to_csv(dataframe, file_path):
    """데이터를 CSV 파일에 추가로 저장"""
    if not dataframe.empty:
        dataframe.to_csv(file_path, mode='a', header=not os.path.exists(file_path), index=False)

# 개별 트랜잭션 해시를 처리하는 함수 (병렬 처리 대상)
def process_transaction(txid, output_file):
    try:
        balance_df = get_transaction_inputs_outputs(txid)
        if balance_df is not None:
            save_to_csv(balance_df, output_file)
            print(f"트랜잭션 '{txid}' 데이터 저장 완료.")
        else:
            print(f"트랜잭션 '{txid}' 데이터가 비어 있습니다.")
            return (txid, "Unprocessed")
    except Exception as e:
        print(f"트랜잭션 처리 오류: {e}, txid: {txid}")
        return (txid, "Failed")

    return (txid, "Processed")

# 파일을 처리하는 함수
def process_file(input_file, output_file):
    error_file = output_file.replace(".csv", "_errors.csv")

    try:
        # 파일 읽기
        df = pd.read_csv(input_file)

        # 'hash' 컬럼이 있는지 확인
        if 'hash' not in df.columns:
            print(f"파일 '{input_file}'에 'hash' 컬럼이 없습니다.")
            return

        txids = df['hash'].tolist()
        print(f"파일 '{input_file}'에서 트랜잭션 해시 추출 완료.")

        # 병렬 처리
        results = []
        with ThreadPoolExecutor(max_workers=8) as executor:  # 병렬 작업 스레드 수 조정
            futures = {executor.submit(process_transaction, txid, output_file): txid for txid in txids}
            for future in futures:
                txid = futures[future]
                try:
                    result = future.result()
                    if result:
                        results.append(result)
                except Exception as e:
                    print(f"트랜잭션 '{txid}' 처리 중 예외 발생: {e}")
                    results.append((txid, "Failed"))

        # 오류 트랜잭션 저장
        error_data = [res for res in results if res[1] != "Processed"]
        if error_data:
            error_df = pd.DataFrame(error_data, columns=["Txid", "Status"])
            error_df.to_csv(error_file, index=False)
            print(f"오류 발생 트랜잭션 저장 완료: {error_file}")

        print(f"병렬 처리 완료. 최종 데이터 저장 완료: {output_file}")

    except Exception as e:
        print(f"파일 읽기 오류: {e}, 파일: {input_file}")

# 실행
input_file = "C:/python/(1-1)모든 transactions.csv"
output_file = "C:/python/(2-0)병렬처리.csv"
process_file(input_file, output_file)


In [ ]:
import os
import pandas as pd
from concurrent.futures import ThreadPoolExecutor

# 원본 파일과 새로운 파일 경로
source_file = "C:/python/(2-0)병렬처리.csv"
new_file = "C:/python/(2-1-0)소형 데이터 제거.csv"

# 파일 처리 함수
def process_file(source_file, new_file):
    if not os.path.exists(source_file):
        print(f"원본 파일 '{source_file}'가 존재하지 않습니다.")
        return

    try:
        # CSV 파일 읽기 (구분자 명시 + 문제 있는 행 스킵)
        df = pd.read_csv(source_file, sep=",", low_memory=False, on_bad_lines="skip")

        # 'value' 컬럼이 있는지 확인
        if 'value' not in df.columns:
            print("'value' 컬럼이 존재하지 않습니다.")
            return

        # 'value' 컬럼을 숫자로 변환, 변환할 수 없는 값은 NaN으로 처리
        df['value'] = pd.to_numeric(df['value'], errors='coerce')

        # 'value' 컬럼 기준으로 값이 10 이상인 행만 필터링
        filtered_df = df[df['value'] > 10]

        # 새로운 파일에 저장
        filtered_df.to_csv(new_file, index=False)
        print(f"파일 '{new_file}'이 저장되었습니다.")
    except Exception as e:
        print(f"파일 '{source_file}' 처리 중 오류 발생: {e}")

# 병렬 처리 함수
def run():
    with ThreadPoolExecutor() as executor:
        executor.submit(process_file, source_file, new_file)

# 실행
run()

print("파일 처리 완료!")


In [ ]:
import os
import pandas as pd
from concurrent.futures import ThreadPoolExecutor

# 파일 경로 설정
input_file = 'C:/python/(2-1-0)소형 데이터 제거.csv'
output_file = 'C:/python/(2-1-1)address 기준 정렬.csv'

# 파일 처리 함수
def process_file(input_file, output_file):
    if not os.path.exists(input_file):
        print(f"원본 파일 '{input_file}'가 존재하지 않습니다.")
        return

    try:
        # CSV 파일 읽기
        df = pd.read_csv(input_file)

        # 'address' 열을 기준으로 1순위, 'date' 열을 기준으로 2순위로 정렬
        df_sorted = df.sort_values(by=['address', 'date'])

        # 정렬된 데이터를 새로운 파일로 저장
        df_sorted.to_csv(output_file, index=False)
        print(f"파일이 '{output_file}'로 저장되었습니다.")
    except Exception as e:
        print(f"파일 '{input_file}' 처리 중 오류 발생: {e}")

# 병렬 처리 함수
def run():
    with ThreadPoolExecutor() as executor:
        executor.submit(process_file, input_file, output_file)

# 실행
run()

print("파일 처리 완료!")


In [ ]:
import pandas as pd
import concurrent.futures
import re

# 파일 경로 설정
input_file = r"C:/python/(2-1-1)address 기준 정렬.csv"

# 'date' 형식을 변환하는 함수
def format_date(value):
    value = str(value)
    # 정규 표현식 적용하여 HH:00:00 형태로 변환
    value = re.sub(r"^(\d{4}-\d{2}-\d{2} \d{2})$", r"\1:00:00", value)
    formatted_value = pd.to_datetime(value, errors='coerce')
    return formatted_value if not pd.isna(formatted_value) else value

# 파일 내 'date' 열 변환 함수
def transform_date_column(input_file):
    print("📥 데이터 로딩 중...")
    df = pd.read_csv(input_file, dtype={'date': 'string'})  # date 열을 문자열로 불러오기

    print("🚀 'date' 변환 시작 (병렬 처리)")
    with concurrent.futures.ProcessPoolExecutor(max_workers=6) as executor:
        df['date'] = list(executor.map(format_date, df['date'], chunksize=10000))

    print("💾 변환 완료! 파일 저장 중...")
    df.to_csv(input_file, index=False)  # 덮어쓰기

    print(f"✅ 'date' 변환 완료 및 저장 완료: {input_file}")

# ✅ Windows에서 병렬 처리 시 필수적인 구조
if __name__ == '__main__':
    transform_date_column(input_file)


In [ ]:
import pandas as pd

# 파일 경로 설정
input_file = r"C:/python/(2-1-1)address 기준 정렬.csv"
btc_prices_file = r"C:\python\BTC_15m_data-datetime.csv"

# BTC 가격 데이터 로드
btc_prices_df = pd.read_csv(btc_prices_file)
btc_prices_df['timestamp'] = pd.to_datetime(btc_prices_df['timestamp']).dt.floor('15min')

def update_price_in_file(input_file):
    # 필터링된 데이터 파일 로드
    filtered_df = pd.read_csv(input_file)
    filtered_df['date'] = pd.to_datetime(filtered_df['date']).dt.floor('15min')  # 15분 단위로 변환

    # 🔥 Merge 활용 (빠름)
    merged_df = filtered_df.merge(btc_prices_df[['timestamp', 'open']],
                                  left_on='date', right_on='timestamp',
                                  how='left')

    # 'price' 열 업데이트
    merged_df.rename(columns={'open': 'price'}, inplace=True)
    merged_df.drop(columns=['timestamp'], inplace=True)

    # 업데이트된 파일 저장
    output_file_path = input_file.replace(".csv", "_updated.csv")
    merged_df.to_csv(output_file_path, index=False)
    print(f"✅ 'price' 열이 15분 단위로 업데이트되어 저장되었습니다: {output_file_path}")

# 실행
update_price_in_file(input_file)


In [ ]:
import pandas as pd

# CSV 파일 읽기 (date 컬럼을 날짜형으로 파싱)
file_path = "C:/python/(2-1-1)address 기준 정렬_updated.csv"
df = pd.read_csv(file_path, parse_dates=['date'])

# 조건: "withdrawal → deposit" 또는 "deposit → withdrawal"
cond_type = ((df['type'] == 'withdrawal') & (df['type'].shift(-1) == 'deposit')) | \
            ((df['type'] == 'deposit') & (df['type'].shift(-1) == 'withdrawal'))

# 날짜 차이 2분 이하
cond_date = (df['date'].shift(-1) - df['date']).abs() <= pd.Timedelta(minutes=2)

# 가격 동일
cond_price = df['price'] == df['price'].shift(-1)

# 모든 조건을 만족하는 경우, 해당 행과 다음 행 모두 삭제
mask = cond_type & cond_date & cond_price
idx = df.index[mask]
indices_to_drop = idx.union(idx + 1)

df_filtered = df.drop(indices_to_drop).reset_index(drop=True)

# 결과 저장
df_filtered.to_csv("C:/python/(2-1-1)address 기준 정렬_updated_filtered.csv", index=False)

print("조건에 해당하는 행이 제거되었습니다.")


In [ ]:
import pandas as pd
from datetime import timedelta
from concurrent.futures import ProcessPoolExecutor

def process_address_group(group):
    local_to_delete = set()  # set으로 중복을 방지
    group_sorted = group.sort_values(by='date')  # 거래일 기준 정렬

    total_transactions = len(group_sorted)  # 전체 거래 수

    # 30일 내 거래를 미리 계산 (중복 방지)
    date_range_dict = {}
    for i in range(len(group_sorted)):
        current_time = group_sorted.iloc[i]['date']
        # 30일 내 거래를 이미 미리 계산해서 저장
        time_window = group_sorted[(group_sorted['date'] >= current_time - timedelta(days=30)) &
                                   (group_sorted['date'] <= current_time)]
        date_range_dict[current_time] = len(time_window)

    # 30일 범위 내의 거래 수를 재사용하여 불필요한 계산을 방지
    for i in range(len(group_sorted)):
        current_time = group_sorted.iloc[i]['date']
        count_in_window = date_range_dict[current_time]  # 미리 계산된 값 사용

        # 70%가 30일 내에 몰려있으면 해당 행 삭제
        if count_in_window / total_transactions >= 0.7:
            local_to_delete.add(group_sorted.index[i])  # set에 추가하여 중복 제거

    return local_to_delete

def main():
    # CSV 파일 불러오기
    file_path = "C:/python/(2-1-1)address 기준 정렬_updated_filtered.csv"
    df = pd.read_csv(file_path)

    # 'date' 컬럼을 datetime 형식으로 변환 (이미 datetime 형식이라면 이 과정은 생략 가능합니다)
    df['date'] = pd.to_datetime(df['date'])

    # 삭제할 행을 담을 리스트
    to_delete = set()

    # ProcessPoolExecutor를 사용하여 병렬 처리
    with ProcessPoolExecutor() as executor:
        # 'address'별로 그룹화한 뒤 병렬 처리
        results = list(executor.map(process_address_group, [group for _, group in df.groupby('address')]))

    # 결과를 합침
    for result in results:
        to_delete.update(result)

    # 삭제할 행 제거
    df_cleaned = df.drop(index=to_delete)

    # 결과를 새로운 파일로 저장
    df_cleaned.to_csv("C:/python/(2-2-2)cleaned.csv", index=False)

# 메인 모듈로 실행될 때만 병렬 처리 시작
if __name__ == '__main__':
    main()


In [ ]:
import pandas as pd
import multiprocessing as mp

# 파일 경로 설정
input_file = r'C:/python/(2-2-2)cleaned.csv'
output_file = r'C:/python/(3-0-0) type_num 부여 + aver_price 구하기.csv'

def assign_type_num(group):
    """type_num 할당"""
    group = group.copy().reset_index(drop=True)

    # 🔹 type_num 할당 (type 변경 시마다 증가)
    group['type_change'] = (group['type'] != group['type'].shift()).astype(int)
    group['type_num'] = group['type_change'].cumsum()
    group.drop(columns=['type_change'], inplace=True)

    # 🔹 aver_price 계산 (type_num 별 평균 계산)
    group['aver_price'] = group.groupby('type_num')['price'].transform('mean')

    return group

def process_data_parallel(df, func):
    """각 address별 데이터를 병렬로 처리"""
    with mp.Pool(processes=mp.cpu_count()) as pool:
        results = pool.map(func, [group for _, group in df.groupby('address')])
    return pd.concat(results)

if __name__ == '__main__':
    # CSV 파일 불러오기
    df = pd.read_csv(input_file)

    # address와 date를 기준으로 정렬
    df = df.sort_values(by=["address", "date"])

    # 🔹 병렬 처리로 type_num과 aver_price 계산
    df = process_data_parallel(df, assign_type_num)

    # 결과 CSV 저장
    df.to_csv(output_file, index=False, encoding="utf-8-sig")

    print(f"✅ 처리 완료! 결과 파일이 저장되었습니다: {output_file}")


In [ ]:
import pandas as pd
import os

# 파일 경로 설정
input_file = 'C:/python/(3-0-0) type_num 부여 + aver_price 구하기.csv'
output_file = 'C:/python/(3-0-1) type_num 기준으로 병합.csv'

# 파일이 존재하는지 확인
if os.path.exists(input_file):
    # CSV 파일 로드
    df = pd.read_csv(input_file)

    # address와 type_num을 기준으로 정렬
    df_sorted = df.sort_values(by=['address', 'type_num'])

    # address별로 동일한 type_num 행을 병합
    # 여기서 type_num을 그냥 첫번째 값으로 가져오는 방식으로 수정
    df_merged = df_sorted.groupby(['address', 'type_num'], as_index=False).agg(
        {'value': 'sum',
         'type': 'first',
         'price': 'first',
         'aver_price': 'first'}).reset_index(drop=True)

    # 병합된 데이터를 새로운 파일로 저장
    if not os.path.exists(output_file):
        df_merged.to_csv(output_file, index=False)
    else:
        df_merged.to_csv(output_file, index=False)

    print("작업이 완료되었습니다.")
else:
    print("입력 파일이 존재하지 않습니다.")


In [ ]:
import pandas as pd
import os
from concurrent.futures import ProcessPoolExecutor

# LONG, SHORT 구분 작업을 수행하는 함수
def process_address(address_group):
    address, group = address_group
    # 첫 번째 행이 withdrawal이면 해당 address는 무시
    if group.iloc[0]['type'] == 'withdrawal':
        return None

    # deposit과 withdrawal을 구분
    deposit_data = group[group['type'] == 'deposit']
    withdrawal_data = group[group['type'] == 'withdrawal']

    if not deposit_data.empty and not withdrawal_data.empty:
        # deposit의 aver_price와 withdrawal의 aver_price 비교
        deposit_aver_price = deposit_data['aver_price'].iloc[0]
        withdrawal_aver_price = withdrawal_data['aver_price'].iloc[0]

        # LONG과 SHORT 계산
        if deposit_aver_price < withdrawal_aver_price:
            result = 'LONG'
        else:
            result = 'SHORT'

        return {'address': address, 'result': result}

def main():
    # 파일 경로 설정
    input_file = 'C:/python/(3-0-1) type_num 기준으로 병합.csv'
    output_file = 'C:/python/(3-0-2) (승률) address 롱 숏 분류.csv'

    # CSV 파일 로드
    df = pd.read_csv(input_file)

    # address별로 데이터 그룹화
    address_groups = df.groupby('address')

    # 병렬 처리로 각 address 처리
    with ProcessPoolExecutor() as executor:
        results = list(executor.map(process_address, address_groups))

    # None을 제외한 결과만 필터링
    results = [result for result in results if result is not None]

    # 결과 DataFrame 생성
    result_df = pd.DataFrame(results)

    # address별로 LONG, SHORT 비율 계산
    final_results = []

    for address, group in result_df.groupby('address'):
        long_count = (group['result'] == 'LONG').sum()
        short_count = (group['result'] == 'SHORT').sum()

        # LONG:SHORT 비율이 6:4 이상이면 LONG, 반대면 SHORT
        if long_count / (long_count + short_count) >= 0.6:
            final_results.append({'address': address, 'prediction': 'LONG'})
        else:
            final_results.append({'address': address, 'prediction': 'SHORT'})

    # 최종 결과 DataFrame 생성
    final_df = pd.DataFrame(final_results)

    # 파일로 저장
    final_df.to_csv(output_file, index=False)

    print("작업이 완료되었습니다.")

# 이 코드가 직접 실행될 때만 main()을 호출
if __name__ == '__main__':
    main()


In [ ]:
import pandas as pd
from concurrent.futures import ProcessPoolExecutor

# CSV 파일 경로
file_path = 'C:/python/(2-2-2)cleaned.csv'
output_path = 'C:/python/(3-1-1) (순수익) long short 분류.csv'

# 데이터 로드
df = pd.read_csv(file_path)

# deposit과 withdrawal 별로 value * price 계산
df['amount'] = df['value'] * df['price']

def classify_long_short(group):
    a = group.loc[group['type'] == 'deposit', 'amount'].sum()
    b = group.loc[group['type'] == 'withdrawal', 'amount'].sum()

    long_short_type = 'LONG' if a < b else 'SHORT'


    return long_short_type

def process_address_group(address_group):
    address, group = address_group
    long_short_type = classify_long_short(group)
    return address, long_short_type  # return a tuple

# 각 address별 LONG_SHORT_TYPE 계산 (병렬 처리)
def main():
    with ProcessPoolExecutor() as executor:
        results = list(executor.map(process_address_group, df.groupby('address')))

    # 결과를 DataFrame으로 변환
    result_df = pd.DataFrame(results, columns=['address', 'LONG_SHORT_TYPE'])

    # 결과 저장
    result_df.to_csv(output_path, index=False)

    print(f"결과 파일이 저장되었습니다: {output_path}")

# 메인 모듈로 실행될 때만 병렬 처리 시작
if __name__ == '__main__':
    main()


In [ ]:
import os
import pandas as pd

# 입력 파일 경로 설정
input_file = 'C:/python/(3-0-0) type_num 부여 + aver_price 구하기.csv'  # 단일 파일 경로

# 결과를 저장할 CSV 파일 경로 설정 (하나의 파일에 저장)
output_file_path = 'C:/python/(3-2)whale_type.csv'

# 첫 번째 파일에서 데이터가 없으면 새로 생성 (헤더 포함)
is_first_file = not os.path.exists(output_file_path)

try:
    # 파일을 읽어 DataFrame 생성
    df = pd.read_csv(input_file)

    # address별로 value의 평균 계산
    address_value_avg = df.groupby('address')['value'].mean()

    # whale_type 분류
    whale_type = []
    for avg_value in address_value_avg:
        if 10 <= avg_value < 100:
            whale_type.append('small')
        elif 100 <= avg_value < 500:
            whale_type.append('medium')
        else:
            whale_type.append('large')

    # 새로운 DataFrame 생성: address와 whale_type
    whale_df = pd.DataFrame({
        'address': address_value_avg.index,
        'whale_type': whale_type
    })

    # 파일에 데이터를 추가 (첫 번째 파일이면 헤더 포함)
    whale_df.to_csv(output_file_path, mode='a', header=is_first_file, index=False)
    print(f"파일 '{input_file}' 처리 완료!")

except Exception as e:
    print(f"파일 '{input_file}' 처리 중 오류 발생: {e}")

print("파일 처리 완료!")


In [ ]:
import os
import pandas as pd

# 입력 파일 경로 (단일 파일)
input_file = "C:/python/(3-0-0) type_num 부여 + aver_price 구하기.csv"
result_file_path = "C:/python/(3-3)trader_type_analysis.csv"

# 거래 주기 분류 함수
def analyze_trader_type(df):
    # 홀수 (시작 거래)와 짝수 (끝 거래) 날짜 추출
    odd_dates = df[df['type_num'] % 2 == 1]['date']
    even_dates = df[df['type_num'] % 2 == 0]['date']

    # 날짜 차이 계산
    date_diffs = []
    min_length = min(len(odd_dates), len(even_dates))  # 홀수와 짝수 날짜 중 더 작은 길이

    for i in range(min_length):
        try:
            # 홀수 날짜와 짝수 날짜 차이 계산 (일 단위)
            date_diff = (even_dates.iloc[i] - odd_dates.iloc[i]).days
            date_diffs.append(date_diff)
        except IndexError:
            continue  # 인덱스 오류가 나면 넘어가기

    # 날짜 차이의 평균 계산
    avg_trade_duration = sum(date_diffs) / len(date_diffs) if date_diffs else 0

    # 거래 주기 분류
    if avg_trade_duration <= 1:
        trader_type = 'scalper'
    elif 1 < avg_trade_duration <= 14:
        trader_type = 'short swing'
    elif 15 <= avg_trade_duration <= 30:
        trader_type = 'long swing'
    else:
        trader_type = 'long term'

    return trader_type

# 개별 파일을 처리하는 함수
def process_file(file_path):
    df = pd.read_csv(file_path)
    df['date'] = pd.to_datetime(df['date']).dt.date

    result_list = []

    for address, group in df.groupby('address'):  # 주소별 그룹화
        trader_type = analyze_trader_type(group)  # 개별 address의 거래 주기 분석
        result_list.append({'address': address, 'trader_type': trader_type})  # 결과 저장

    result_df = pd.DataFrame(result_list)  # 최종 데이터프레임 생성
    return result_df


# 결과 파일 생성 함수
def create_trader_type_file():
    # 단일 파일을 처리
    result_df = process_file(input_file)

    # 결과를 CSV로 저장
    result_df.to_csv(result_file_path, index=False)
    print(f"✅ 트레이더 유형 분석 완료! 결과 파일: {result_file_path}")

# 실행
if __name__ == "__main__":
    create_trader_type_file()


In [ ]:
import pandas as pd

# 파일 경로
input_file_path = "C:/python/(3-3)trader_type_analysis.csv"
output_file_path = "C:/python/(3-4)trader_type_analysis_most.csv"

# 데이터 불러오기
df = pd.read_csv(input_file_path)

# address별로 가장 많이 나온 trader_type을 추출
most_common_trader_type = df.groupby('address')['trader_type'].agg(lambda x: x.mode()[0]).reset_index()

# 결과를 새로운 CSV로 저장
most_common_trader_type.to_csv(output_file_path, index=False)

print(f"✅ address별로 가장 많이 나온 trader_type이 추출된 결과 파일이 저장되었습니다: {output_file_path}")


In [ ]:
import os
import pandas as pd

# 파일 경로 수정 (C:\python 경로에 있는 파일들)
long_short_file_1_path = "C:/python/(3-0-2) (승률) address 롱 숏 분류.csv"
long_short_file_2_path = "C:/python/(3-1-1) (순수익) long short 분류.csv"
trader_type_file_path = "C:/python/(3-4)trader_type_analysis_most.csv"
whale_type_file_path = "C:/python/(3-2)whale_type.csv"

# 각 파일을 불러오기
long_short_1_df = pd.read_csv(long_short_file_1_path)
long_short_2_df = pd.read_csv(long_short_file_2_path)
trader_type_df = pd.read_csv(trader_type_file_path)
whale_type_df = pd.read_csv(whale_type_file_path)

# 각 파일을 address 기준으로 병합
merged_df = pd.merge(long_short_1_df, long_short_2_df, on='address', how='outer')
merged_df = pd.merge(merged_df, trader_type_df, on='address', how='outer')
merged_df = pd.merge(merged_df, whale_type_df, on='address', how='outer')

# '|'a-b|' 컬럼 삭제
if '|a-b|' in merged_df.columns:
    merged_df.drop(columns='|a-b|', inplace=True)

# 결과 출력 (옵션)
print(merged_df.head())

# 결과를 새로운 파일로 저장
merged_df.to_csv('C:/python/(3-5)각각의 결과파일.csv', index=False)


In [ ]:
import pandas as pd

# 기존 파일 경로
merged_file_path = "C:/python/(3-5)각각의 결과파일.csv"
output_file_path = "C:/python/(3-6)prediction, LONG_SHORT_TYPE 일치.csv"

# 파일 불러오기
merged_df = pd.read_csv(merged_file_path)

# prediction과 LONG_SHORT_TYPE이 일치하는 address만 필터링
filtered_df = merged_df[merged_df['prediction'] == merged_df['LONG_SHORT_TYPE']]

# 필요한 컬럼만 추출
final_df = filtered_df[['address', 'prediction', 'trader_type', 'whale_type']]

# 결과를 새로운 파일로 저장
final_df.to_csv(output_file_path, index=False)

print(f"✅ 필터링된 결과가 '{output_file_path}'로 저장되었습니다!")


In [ ]:
import os
import pandas as pd
import aiohttp
import asyncio
from datetime import datetime

# API 정보
BITCORE_API_URL = "https://api.bitcore.io/api/BTC/mainnet/address/"
DATA_FILE = "C:/python/(3-6)prediction, LONG_SHORT_TYPE 일치.csv"
OUTPUT_FILE = "C:/python/(4-0) 최근 블록 조회.csv"
MAX_CONCURRENT_REQUESTS = 10  # 동시 요청 개수
MAX_ROWS = 31296  # 최대 처리 행 수

# API 요청 함수 (재시도 횟수 제한)
async def fetch_transaction(session, address, retries=3):
    url = BITCORE_API_URL + address + "/txs"
    try:
        async with session.get(url) as response:
            if response.status == 200:
                transactions = await response.json()
                if transactions:
                    latest_tx = transactions[0]
                    is_deposit = any(address in vin.get("addr", "") for vin in latest_tx.get("vin", []))
                    print(f"Success: {address} - {('deposit' if is_deposit else 'withdrawal')}")
                    return {"address": address, "transaction_type": "deposit" if is_deposit else "withdrawal"}

            elif response.status == 429 and retries > 0:
                print(f"Rate Limit Exceeded for {address}. Retrying in 5 seconds... ({retries} retries left)")
                await asyncio.sleep(5)
                return await fetch_transaction(session, address, retries - 1)

            else:
                print(f"Failed: {address} - Status code {response.status}")
    except Exception as e:
        print(f"Error: {address} - {str(e)}")

    return {"address": address, "transaction_type": None}

# 주소 리스트에서 데이터를 가져오는 함수
async def fetch_all_transactions(addresses):
    address_data = []
    async with aiohttp.ClientSession() as session:
        tasks = []
        for address in addresses:
            tasks.append(fetch_transaction(session, address))
            if len(tasks) >= MAX_CONCURRENT_REQUESTS:
                results = await asyncio.gather(*tasks)
                update_results(results)
                tasks = []
                if check_row_limit():
                    return
        if tasks:
            results = await asyncio.gather(*tasks)
            update_results(results)

# 기존 데이터 업데이트 함수
def update_results(new_data):
    new_df = pd.DataFrame(new_data)
    if os.path.exists(OUTPUT_FILE):
        try:
            df_output = pd.read_csv(OUTPUT_FILE)
        except pd.errors.EmptyDataError:
            df_output = pd.DataFrame(columns=["address", "transaction_type"])
    else:
        df_output = pd.DataFrame(columns=["address", "transaction_type"])

    if not new_df.empty and "address" in new_df.columns:
        df_output = df_output.set_index("address")
        new_df = new_df.set_index("address")
        df_output.update(new_df)
        df_output = df_output.reset_index()

    df_output.to_csv(OUTPUT_FILE, index=False)
    print(f"Updated {len(new_df)} records. Total: {len(df_output)}")

# 현재 행 수 확인 후 최대값 초과 시 중단
def check_row_limit():
    if os.path.exists(OUTPUT_FILE):
        df_output = pd.read_csv(OUTPUT_FILE)
        if len(df_output) >= MAX_ROWS:
            print(f"Reached max rows ({MAX_ROWS}). Stopping process.")
            return True
    return False

# 주소 리스트 불러오기
df = pd.read_csv(DATA_FILE)
addresses = set(df['address'].dropna().unique()) if 'address' in df.columns else set()
print(f"Total addresses to process: {len(addresses)}")

# 실행 시작 시간 기록
start_time = datetime.now()
print(f"Process started at {start_time.strftime('%Y-%m-%d %H:%M:%S')}")

# 비동기 실행
asyncio.run(fetch_all_transactions(addresses))

# 실행 완료 시간 기록
end_time = datetime.now()
elapsed_time = end_time - start_time
hours, remainder = divmod(elapsed_time.total_seconds(), 3600)
minutes, seconds = divmod(remainder, 60)
formatted_time = f"{int(hours)}h {int(minutes)}m {int(seconds)}s"

# 종료 시간 출력
print(f"Process completed at {end_time.strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Total time taken: {formatted_time}")


In [ ]:
import os
import re
import pandas as pd
import requests
import concurrent.futures
from datetime import datetime

# 폴더 및 파일 경로 설정
transactions_folder = "C:/python/python(1-0)TSV"
recent_blocks_file = "C:/python/(4-0) 최근 블록 조회.csv"

# 폴더 내 파일 목록을 날짜순으로 정렬
def get_sorted_transaction_files():
    files = [f for f in os.listdir(transactions_folder) if f.endswith(".tsv")]

    # 파일명에서 날짜 추출 (예: blockchair_bitcoin_transactions_20250216.tsv → 20250216)
    def extract_date(filename):
        match = re.search(r"(\d{8})", filename)
        return int(match.group(1)) if match else 0

    # 날짜순 정렬
    sorted_files = sorted(files, key=extract_date)

    return [os.path.join(transactions_folder, f) for f in sorted_files]

# 10BTC 이상인 트랜잭션 필터링
def filter_large_transactions(file_path):
    df = pd.read_csv(file_path, sep='\t')  # TSV 파일 읽기
    df_filtered = df[df['input_total'] >= 1_000_000_000]  # 10 BTC 이상
    return df_filtered['hash'].tolist()

# 트랜잭션 정보 가져오기 (API 요청)
def fetch_transaction(txid):
    url = f"https://blockchain.info/rawtx/{txid}"
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        return response.json()
    except requests.RequestException:
        print(f"{get_current_timestamp()} ❌ 트랜잭션 {txid} 처리 실패 (API 요청 오류)")
        return None

# 병렬로 트랜잭션 데이터 가져오기 (비동기 방식)
def get_transactions_data(txids):
    transactions = {}
    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        futures = {executor.submit(fetch_transaction, txid): txid for txid in txids}

        for future in concurrent.futures.as_completed(futures):
            txid = futures[future]
            try:
                data = future.result()
                if data:
                    transactions[txid] = data
            except Exception as e:
                print(f"{get_current_timestamp()} ❌ 트랜잭션 {txid} 처리 실패 (예외 발생: {e})")

    return transactions

# 병렬로 주소 매칭 및 입출금 판별 (배치 처리)
def process_address_batch(addresses_set, tx_data_batch):
    results = []
    for tx in tx_data_batch:
        matched_type = None
        inputs = {inp['prev_out']['addr'] for inp in tx.get('inputs', []) if 'prev_out' in inp}
        outputs = {out['addr'] for out in tx.get('out', []) if 'addr' in out}

        if any(address in outputs for address in addresses_set):
            matched_type = "deposit"
        elif any(address in inputs for address in addresses_set):
            matched_type = "withdrawal"

        results.append(matched_type)
    return results

# 병렬로 트랜잭션 배치 처리
def process_transaction_batches(addresses_set, tx_data, batch_size=500):
    with concurrent.futures.ThreadPoolExecutor(max_workers=20) as executor:
        futures = []
        for i in range(0, len(tx_data), batch_size):
            tx_batch = tx_data[i:i + batch_size]
            futures.append(executor.submit(process_address_batch, addresses_set, tx_batch))

        all_results = []
        for future in concurrent.futures.as_completed(futures):
            result = future.result()
            all_results.extend(result)

    return all_results

# 현재 시간 가져오기
def get_current_timestamp():
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

# 메인 실행 로직
def main():
    print(f"{get_current_timestamp()} 🔄 트랜잭션 파일 정렬 중...")
    transaction_files = get_sorted_transaction_files()

    for file in transaction_files:
        print(f"{get_current_timestamp()} 📂 처리 중: {file}")

        txids = filter_large_transactions(file)
        print(f"{get_current_timestamp()} 🚀 API 병렬 요청 실행 중...")
        transactions_data = get_transactions_data(txids)

        print(f"{get_current_timestamp()} 📂 최근 1일 트랜잭션 CSV 파일 로드 중...")
        df_recent = pd.read_csv(recent_blocks_file)

        addresses_set = set(df_recent['address'])  # 집합으로 변환
        print(f"{get_current_timestamp()} 📜 총 {len(addresses_set)}개의 지갑 주소 로드 완료.")

        print(f"{get_current_timestamp()} 🔎 주소 매칭 및 입출금 판별 중...")

        all_results = process_transaction_batches(addresses_set, list(transactions_data.values()), batch_size=500)

        for i, result in enumerate(all_results):
            if result:
                df_recent.at[i, 'transaction_type'] = result

            if (i+1) % 1000 == 0:
                print(f"{get_current_timestamp()} {i+1}/{len(df_recent)} 주소 매칭 처리 중...")

        print(f"{get_current_timestamp()} 💾 파일 업데이트 중...")
        df_recent.to_csv(recent_blocks_file, index=False)
        print(f"{get_current_timestamp()} ✅ 처리 완료! 기존 파일이 업데이트됨: {recent_blocks_file}")

if __name__ == "__main__":
    main()


In [ ]:
import asyncio
import aiohttp
import csv
import json
import requests
import os
import pandas as pd
from pathlib import Path
from datetime import datetime

# Bitcoin Core RPC 설정
RPC_URL = "http://localhost:8332"
HEADERS = {"content-type": "application/json"}
COOKIE_PATH = Path("C:/Users/fhqhz/AppData/Roaming/Bitcoin/.cookie")
CSV_FILE = Path("C:/python/(4-0) 최근 블록 조회.csv")
BLOCKCHAIN_INFO_API = "https://blockchain.info/rawtx/{}"

def log_time(message):
    now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    print(f"[{now}] {message}")

def get_rpc_auth():
    with open(COOKIE_PATH, "r") as f:
        return tuple(f.read().strip().split(":", 1))

def get_latest_blocks(count=30):
    log_time(f"최근 {count}개 블록 조회 시작")
    auth = get_rpc_auth()

    payload = json.dumps({"jsonrpc": "1.0", "id": "curltest", "method": "getbestblockhash", "params": []})
    response = requests.post(RPC_URL, headers=HEADERS, auth=auth, data=payload)

    if response.status_code != 200:
        log_time(f"getbestblockhash 호출 실패: {response.status_code}")
        return []

    latest_block_hash = response.json().get("result")
    if latest_block_hash is None:
        log_time("getbestblockhash 결과가 None임")
        return []

    blocks = []
    for _ in range(count):
        payload = json.dumps({"jsonrpc": "1.0", "id": "curltest", "method": "getblock", "params": [latest_block_hash, 2]})
        response = requests.post(RPC_URL, headers=HEADERS, auth=auth, data=payload)

        if response.status_code != 200:
            log_time(f"getblock 호출 실패: {response.status_code}")
            break

        block_data = response.json().get("result")
        if block_data is None:
            log_time("getblock 응답에서 result가 None임")
            break

        blocks.append(block_data)
        latest_block_hash = block_data.get("previousblockhash")
        if not latest_block_hash:
            break

    log_time(f"최근 {len(blocks)}개 블록 조회 완료")
    return blocks[::-1]

def filter_large_transactions(transactions):
    log_time("10BTC 이상 거래 필터링 시작")
    result = [tx["txid"] for tx in transactions if sum(out["value"] for out in tx.get("vout", [])) >= 10]
    log_time(f"10BTC 이상 거래 {len(result)}건 필터링 완료")
    return result

async def fetch_transaction_data(session, txid):
    try:
        async with session.get(BLOCKCHAIN_INFO_API.format(txid)) as response:
            if response.status != 200:
                log_time(f"트랜잭션 {txid} 조회 실패: {response.status}")
                return None
            return await response.json()
    except Exception as e:
        log_time(f"트랜잭션 {txid} 조회 중 오류 발생: {e}")
        return None

async def get_transaction_addresses(txids):
    log_time("트랜잭션 주소 조회 시작")
    addresses = {}
    async with aiohttp.ClientSession() as session:
        tasks = [fetch_transaction_data(session, txid) for txid in txids]
        results = await asyncio.gather(*tasks)
        for tx_data in results:
            if not tx_data:
                continue  # 조회 실패 시 패스
            if "inputs" in tx_data:
                for inp in tx_data["inputs"]:
                    if "prev_out" in inp and "addr" in inp["prev_out"]:
                        addresses[inp["prev_out"]["addr"]] = "withdrawal"
            if "out" in tx_data:
                for out in tx_data["out"]:
                    if "addr" in out:
                        addresses[out["addr"]] = "deposit"
    log_time("트랜잭션 주소 조회 완료")
    return addresses

def update_csv(address_data):
    log_time("CSV 파일 업데이트 시작")
    rows = []
    with open(CSV_FILE, "r", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            if row["address"] in address_data:
                row["transaction_type"] = address_data[row["address"]]
            rows.append(row)
    with open(CSV_FILE, "w", encoding="utf-8", newline="") as f:
        writer = csv.DictWriter(f, fieldnames=rows[0].keys())
        writer.writeheader()
        writer.writerows(rows)
    log_time("CSV 파일 업데이트 완료")

async def main():
    blocks = get_latest_blocks(30)
    all_txids = []
    for block in blocks:
        all_txids.extend(filter_large_transactions(block.get("tx", [])))

    address_data = await get_transaction_addresses(all_txids)
    update_csv(address_data)
    log_time("작업 완료")

if __name__ == "__main__":
    asyncio.run(main())


In [ ]:
import pandas as pd
import requests
import concurrent.futures
from datetime import datetime

# 파일 경로 설정
transactions_file = "C:/python/python(1-0)TSV/blockchair_bitcoin_transactions_20250309.tsv"
recent_blocks_file = "C:/python/(4-0) 최근 블록 조회.csv"

# 10BTC 이상인 트랜잭션 필터링
def filter_large_transactions():
    df = pd.read_csv(transactions_file, sep='\t')  # TSV 파일 읽기
    df_filtered = df[df['input_total'] >= 1_000_000_000]  # 10 BTC 이상
    return df_filtered['hash'].tolist()

# 트랜잭션 정보 가져오기 (API 요청)
def fetch_transaction(txid):
    url = f"https://blockchain.info/rawtx/{txid}"
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        return response.json()
    except requests.RequestException:
        print(f"{get_current_timestamp()} ❌ 트랜잭션 {txid} 처리 실패 (API 요청 오류)")
        return None

# 병렬로 트랜잭션 데이터 가져오기 (비동기 방식)
def get_transactions_data(txids):
    transactions = {}
    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        futures = {executor.submit(fetch_transaction, txid): txid for txid in txids}

        for future in concurrent.futures.as_completed(futures):
            txid = futures[future]
            try:
                data = future.result()
                if data:
                    transactions[txid] = data
            except Exception as e:
                print(f"{get_current_timestamp()} ❌ 트랜잭션 {txid} 처리 실패 (예외 발생: {e})")

    return transactions

# 주소 매칭 및 입출금 판별
def determine_transaction_type(address, tx_data):
    for tx in sorted(tx_data, key=lambda x: x['time'], reverse=True):  # 최신 트랜잭션 우선
        inputs = {inp['prev_out']['addr'] for inp in tx.get('inputs', []) if 'prev_out' in inp}
        outputs = {out['addr'] for out in tx.get('out', []) if 'addr' in out}

        if address in outputs:
            return "deposit"
        elif address in inputs:
            return "withdrawal"
    return None  # 결과 없음

# 병렬로 주소 매칭 및 입출금 판별 (배치 처리)
def process_address_batch(addresses_set, tx_data_batch):
    results = []
    for tx in tx_data_batch:
        matched_type = None
        # 트랜잭션의 입력과 출력을 집합으로 변환
        inputs = {inp['prev_out']['addr'] for inp in tx.get('inputs', []) if 'prev_out' in inp}
        outputs = {out['addr'] for out in tx.get('out', []) if 'addr' in out}

        # 주소가 출력에 있으면 deposit, 입력에 있으면 withdrawal
        if any(address in outputs for address in addresses_set):
            matched_type = "deposit"
        elif any(address in inputs for address in addresses_set):
            matched_type = "withdrawal"

        results.append(matched_type)
    return results

# 병렬로 트랜잭션 배치 처리
def process_transaction_batches(addresses_set, tx_data, batch_size=500):
    with concurrent.futures.ThreadPoolExecutor(max_workers=20) as executor:
        # 트랜잭션을 배치로 나누어 처리
        futures = []
        for i in range(0, len(tx_data), batch_size):
            tx_batch = tx_data[i:i + batch_size]
            futures.append(executor.submit(process_address_batch, addresses_set, tx_batch))

        # 결과를 처리
        all_results = []
        for future in concurrent.futures.as_completed(futures):
            result = future.result()
            all_results.extend(result)

    return all_results

# 현재 시간 가져오기
def get_current_timestamp():
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

# 메인 실행 로직
def main():
    print(f"{get_current_timestamp()} 🔄 10BTC 이상 트랜잭션 필터링 중...")
    txids = filter_large_transactions()

    print(f"{get_current_timestamp()} 🚀 API 병렬 요청 실행 중...")
    transactions_data = get_transactions_data(txids)

    print(f"{get_current_timestamp()} 📂 최근 1일 트랜잭션 CSV 파일 로드 중...")
    df_recent = pd.read_csv(recent_blocks_file)

    addresses_set = set(df_recent['address'])  # 집합으로 변환
    print(f"{get_current_timestamp()} 📜 총 {len(addresses_set)}개의 지갑 주소 로드 완료.")

    print(f"{get_current_timestamp()} 🔎 주소 매칭 및 입출금 판별 중...")

    # 트랜잭션을 병렬로 처리
    all_results = process_transaction_batches(addresses_set, list(transactions_data.values()), batch_size=500)

    # 결과 반영
    for i, result in enumerate(all_results):
        if result:
            df_recent.at[i, 'transaction_type'] = result

        if (i+1) % 1000 == 0:  # 1000개마다 출력
            print(f"{get_current_timestamp()} {i+1}/{len(df_recent)} 주소 매칭 처리 중...")

    print(f"{get_current_timestamp()} 💾 파일 업데이트 중...")
    df_recent.to_csv(recent_blocks_file, index=False)
    print(f"{get_current_timestamp()} ✅ 처리 완료! 기존 파일이 업데이트됨: ", recent_blocks_file)

if __name__ == "__main__":
    main()


In [ ]:
import asyncio
import aiohttp
import csv
import json
import requests
import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from pathlib import Path
from datetime import datetime

# Bitcoin Core RPC 설정
RPC_URL = "http://localhost:8332"
HEADERS = {"content-type": "application/json"}
COOKIE_PATH = Path("C:/Users/fhqhz/AppData/Roaming/Bitcoin/.cookie")
CSV_FILE = Path("C:/python/(4-0) 최근 블록 조회.csv")
VISUALIZATION_FOLDER = Path("C:/python/(4-1) 시각화/")
VISUALIZATION_FILE = VISUALIZATION_FOLDER / "전체_시각화.png"
BLOCKCHAIN_INFO_API = "https://blockchain.info/rawtx/{}"

# 폴더 자동 생성
VISUALIZATION_FOLDER.mkdir(parents=True, exist_ok=True)

def log_time(message):
    now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    print(f"[{now}] {message}")

def get_rpc_auth():
    with open(COOKIE_PATH, "r") as f:
        return tuple(f.read().strip().split(":", 1))

def get_latest_block():
    log_time("최신 블록 해시 조회 시작")
    auth = get_rpc_auth()
    payload = json.dumps({"jsonrpc": "1.0", "id": "curltest", "method": "getbestblockhash", "params": []})
    response = requests.post(RPC_URL, headers=HEADERS, auth=auth, data=payload)
    log_time("최신 블록 해시 조회 완료")
    return response.json().get("result")

def get_block_transactions(block_hash):
    log_time("블록 트랜잭션 조회 시작")
    auth = get_rpc_auth()
    payload = json.dumps({"jsonrpc": "1.0", "id": "curltest", "method": "getblock", "params": [block_hash, 2]})
    response = requests.post(RPC_URL, headers=HEADERS, auth=auth, data=payload)
    log_time("블록 트랜잭션 조회 완료")
    return response.json().get("result", {}).get("tx", [])

def filter_large_transactions(transactions):
    log_time("10BTC 이상 거래 필터링 시작")
    result = [tx["txid"] for tx in transactions if sum(out["value"] for out in tx.get("vout", [])) >= 10]
    log_time(f"10BTC 이상 거래 {len(result)}건 필터링 완료")
    return result

async def fetch_transaction_data(session, txid):
    url = BLOCKCHAIN_INFO_API.format(txid)
    try:
        async with session.get(url) as response:
            if response.status != 200:
                log_time(f"API 오류 {response.status}: {txid}")
                return None  # 오류 발생 시 None 반환

            if "application/json" not in response.content_type:
                log_time(f"잘못된 응답 타입: {response.content_type} - {txid}")
                return None  # JSON이 아닌 응답 무시

            return await response.json()
    except Exception as e:
        log_time(f"예외 발생: {e} - {txid}")
        return None  # 오류 발생하면 None 반환


async def get_transaction_addresses(txids):
    log_time("트랜잭션 주소 조회 시작")
    addresses = {}
    async with aiohttp.ClientSession() as session:
        tasks = [fetch_transaction_data(session, txid) for txid in txids]
        results = await asyncio.gather(*tasks)
        for tx_data in results:
            if tx_data is None:
                continue  # None 값은 건너뜀
            # 입력 주소 확인
            if "inputs" in tx_data:
                for inp in tx_data["inputs"]:
                    if "prev_out" in inp and "addr" in inp["prev_out"]:
                        addresses[inp["prev_out"]["addr"]] = "withdrawal"
            # 출력 주소 확인
            if "out" in tx_data:
                for out in tx_data["out"]:
                    if "addr" in out:
                        addresses[out["addr"]] = "deposit"
    log_time("트랜잭션 주소 조회 완료")
    return addresses

def update_csv(address_data):
    log_time("CSV 파일 업데이트 시작")

    # 기존 데이터 로드
    rows = []
    changes = {}  # 변경 사항 저장

    with open(CSV_FILE, "r", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            old_type = row["transaction_type"]
            new_type = address_data.get(row["address"], old_type)  # 없으면 기존 값 유지

            if old_type != new_type:
                key = f"{row['prediction']}_{row['trader_type']}_{row['whale_type']}"

                # deposit 증가 ➝ +1, withdrawal 증가 ➝ -1
                if new_type == "deposit":
                    changes[key] = changes.get(key, 0) + 1
                elif new_type == "withdrawal":
                    changes[key] = changes.get(key, 0) - 1

            row["transaction_type"] = new_type
            rows.append(row)

    # CSV 파일 업데이트
    with open(CSV_FILE, "w", encoding="utf-8", newline="") as f:
        writer = csv.DictWriter(f, fieldnames=rows[0].keys())
        writer.writeheader()
        writer.writerows(rows)

    log_time("CSV 파일 업데이트 완료")



    # 변경 사항 출력
    if changes:
        for key, change in changes.items():
            change_sign = "+" if change > 0 else ""
            print(f"{key} : {change_sign}{change}")
    else:
        print("🔹 변경된 데이터 없음")



def visualize_data():
    log_time("데이터 시각화 시작")
    df = pd.read_csv(CSV_FILE, encoding="utf-8-sig")
    trader_type_order = ['scalper', 'short swing', 'long swing', 'long term']
    whale_type_order = ['small', 'medium', 'large']
    df['trader_type'] = pd.Categorical(df['trader_type'], categories=trader_type_order, ordered=True)
    df['whale_type'] = pd.Categorical(df['whale_type'], categories=whale_type_order, ordered=True)
    df['prediction'] = df['prediction'].apply(lambda x: 'LONG' if x == 'LONG' else 'SHORT')
    df.sort_values(by=['prediction', 'whale_type', 'trader_type'], ascending=[False, True, True], inplace=True)
    combinations = df[['prediction', 'trader_type', 'whale_type']].drop_duplicates()
    num_plots = len(combinations)
    cols = 4
    rows = (num_plots + cols - 1) // cols
    fig, axes = plt.subplots(rows, cols, figsize=(10, 1 * rows))
    axes = axes.flatten()
    font_path = "C:/Windows/Fonts/malgun.ttf"
    prop = fm.FontProperties(fname=font_path)
    plt.rcParams['font.family'] = prop.get_name()
    plt.rcParams['axes.unicode_minus'] = False
    for idx, (_, row) in enumerate(combinations.iterrows()):
        if idx >= len(axes):
            break
        prediction, trader_type, whale_type = row
        filtered_data = df[(df['prediction'] == prediction) & (df['trader_type'] == trader_type) & (df['whale_type'] == whale_type)]

        # None 값 제외 (여기 추가)
        filtered_data = filtered_data[filtered_data['transaction_type'] != "Non"]

        if filtered_data.empty:
            continue
        transaction_counts = filtered_data['transaction_type'].value_counts()

        ax = axes[idx]
        transaction_counts = filtered_data['transaction_type'].value_counts()
        color_map = {'deposit': 'green', 'withdrawal': 'red'}
        colors = [color_map.get(tx_type, 'gray') for tx_type in transaction_counts.index]

        transaction_counts.plot(kind='barh', color=colors, ax=ax)


        ax.set_title(f"{prediction}_{trader_type}_{whale_type}", fontsize=10)
        ax.set_xlabel('')
        ax.set_ylabel('')


    for idx in range(len(combinations), len(axes)):
        fig.delaxes(axes[idx])
    plt.tight_layout()
    plt.savefig(VISUALIZATION_FILE, dpi=300)
    plt.close()
    log_time("데이터 시각화 완료")



async def main():
    while True:
        block_hash = get_latest_block()
        transactions = get_block_transactions(block_hash)
        large_txids = filter_large_transactions(transactions)
        address_data = await get_transaction_addresses(large_txids)
        update_csv(address_data)
        visualize_data()
        log_time("대기 중...")
        await asyncio.sleep(480)

if __name__ == "__main__":
    asyncio.run(main())